In [16]:
from ConnectDB import *
import pandas as pd
import pandas_ta as ta
from indicators import add_indicators

In [19]:
# Create connection
con,cur = connect_data_base()

DataBase Connected!
Version: PostgreSQL 14.5 (Homebrew) on aarch64-apple-darwin22.1.0, compiled by Apple clang version 14.0.0 (clang-1400.0.29.202), 64-bit


In [13]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

df = postgresql_to_dataframe(conn=con, select_query="SELECT * FROM cryptodata_2022_11_09", column_names=['symbol', 'timestamp', 'open', 'high', 'low', 'close', 'volume', 'trade_count', 'vwap'])

In [8]:
import pandas as pd
import numpy as np
import streamz
import streamz.dataframe as sdf
import hvplot.streamz
import hvplot.pandas
import holoviews as hv
import datetime
import time
import psycopg2
from ConnectDB import *
from streamz import Stream

In [ ]:
con,cur = connect_data_base()
cur.execute("SELECT * FROM cryptodata_2022_11_09 ORDER BY timestamp DESC LIMIT 60")
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['symbol', 'timestamp', 'open', 'high', 'low', 'close', 'volume', 'trade_count', 'vwap'])
df = df[['timestamp','close']]

stream = Stream()

source = sdf.DataFrame(stream, example=stream)




In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash import dash_table
import pandas as pd

app = dash.Dash(__name__)
cur.execute("SELECT * FROM cryptodata_2022_11_09 ORDER BY timestamp DESC LIMIT 60")
tupples = cur.fetchall()
df = pd.DataFrame(data=tupples, columns=['symbol', 'timestamp', 'open', 'high', 'low', 'close', 'volume', 'trade_count', 'vwap'])

app.layout = html.Div([
      html.H4('Dashboard'),
      dcc.Interval('graph-update', interval = 2000, n_intervals = 0),
      dash_table.DataTable(
          id = 'table',
          data = df.to_dict('records'),
          columns=[{"name": i, "id": i} for i in df.columns])])

@app.callback(
        dash.dependencies.Output('table','data'),
        [dash.dependencies.Input('graph-update', 'n_intervals')])
def updateTable(n):
    """ Query database 
    """
    cur.execute("SELECT * FROM cryptodata_2022_11_09 ORDER BY timestamp DESC LIMIT 60")
    tupples = cur.fetchall()
    df = pd.DataFrame(data=tupples, columns=['symbol', 'timestamp', 'open', 'high', 'low', 'close', 'volume', 'trade_count', 'vwap'])
    return df.to_dict('records')

if __name__ == '__main__':
     app.run_server(debug=True, port=8080)

In [ ]:
import psycopg2
import dash
import dash_core_components as dcc
import dash_html_components as html

con,cur = connect_data_base()
cur.execute("SELECT * FROM cryptodata_2022_11_09")
rows = cur.fetchall()

app = dash.Dash()
app.layout = html.Div([
    html.Table([html.Tr([html.Th(col) for col in rows[0]])] +
               [html.Tr([
                   html.Td(row[col]) for col, row in enumerate(rows)
               ]) for row in rows]
               )
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
df = add_indicators(Data_df = df)